### Student Name: Jose Zacarias
### Studen tID: N01663659

# Lab 03

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [14]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the Fashion MNIST dataset with standardization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Standardizing the input
])

train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=256, shuffle=False)

In [16]:
# neural network definition
class FullyConnectedNN(nn.Module):
    def __init__(self):
        super(FullyConnectedNN, self).__init__()
        
        self.fc1 = nn.Linear(28*28, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.dropout1 = nn.Dropout(0.2)
        
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(0.2)
        
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.dropout3 = nn.Dropout(0.2)
        
        self.fc4 = nn.Linear(64, 10)
        
        # Initialize weights using He initialization
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.kaiming_normal_(self.fc2.weight)
        nn.init.kaiming_normal_(self.fc3.weight)
        nn.init.kaiming_normal_(self.fc4.weight)

    def forward(self, x):
        x = x.view(-1, 28*28)  # Flatten the input images
        
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        
        x = torch.relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)
        
        x = self.fc4(x)
        
        return x

# Initialize the model, loss function, and optimizer
model = FullyConnectedNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
def train_model(model, criterion, optimizer, train_loader, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Evaluate the model
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = correct / total * 100
    print(f'Test Accuracy: {accuracy:.2f}%')

# Train and evaluate
train_model(model, criterion, optimizer, train_loader, num_epochs=20)
evaluate_model(model, test_loader)


Epoch [1/20], Loss: 0.6872
Epoch [2/20], Loss: 0.4358
Epoch [3/20], Loss: 0.3885
Epoch [4/20], Loss: 0.3589
Epoch [5/20], Loss: 0.3392
Epoch [6/20], Loss: 0.3212
Epoch [7/20], Loss: 0.3082
Epoch [8/20], Loss: 0.2945
Epoch [9/20], Loss: 0.2826
Epoch [10/20], Loss: 0.2744
Epoch [11/20], Loss: 0.2611
Epoch [12/20], Loss: 0.2523
Epoch [13/20], Loss: 0.2459
Epoch [14/20], Loss: 0.2429
Epoch [15/20], Loss: 0.2366
Epoch [16/20], Loss: 0.2249
Epoch [17/20], Loss: 0.2217
Epoch [18/20], Loss: 0.2157
Epoch [19/20], Loss: 0.2098
Epoch [20/20], Loss: 0.2060
Test Accuracy: 89.68%
